In [3]:
import pandas as pd
import os
from glob import glob
from collections import Counter
import sqlite3

In [4]:
f='/labs/mmdavis/chen/tcr/bulk_data_3/TRB'

## test

In [10]:
df=pd.read_csv('/labs/mmdavis/chen/tcr/bulk_data_3/TRB/HD_KIR_NEG5.TRB',sep='\t')

In [11]:
df.head(2)

,cloneId,cloneCount,cloneFraction,targetSequences,targetQualities,allVHitsWithScore,allDHitsWithScore,allJHitsWithScore,allCHitsWithScore,allVAlignments,...,nSeqFR4,minQualFR4,aaSeqFR1,aaSeqCDR1,aaSeqFR2,aaSeqCDR2,aaSeqFR3,aaSeqCDR3,aaSeqFR4,refPoints
0,0,3779.0,0.030025,TGTGCCAGCAGCCCATCGGTTTCCTACGAGCAGTACTTC,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC,TRBV9*00(449),NaN,TRBJ2-7*00(228.1),"TRBC2*00(297.6),TRBC1*00(264.1)",270|282|306|0|12||60.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASSPSVSYEQYF,NaN,:::::::::0:-4:12:::::21:-1:39:::
1,1,2959.0,0.023510,TGTGCCACCAGTGCTGGGGAAACTGAAGCTTTCTTT,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC,TRBV24-1*00(479.6),NaN,TRBJ1-1*00(217.3),"TRBC1*00(300.2),TRBC2*00(265.2)",270|283|308|0|13||65.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CATSAGETEAFF,NaN,:::::::::0:-5:13:::::18:-1:36:::


In [51]:
v_list=list(df['allVHitsWithScore'])

count_v=dict()
for i in v_list:
    a=i.split('*')[0]
    if a not in count_v:
        count_v[a]=1
    else:
        count_v[a]+=1

NameError: name 'df' is not defined

In [50]:
pd.DataFrame.from_dict(count_v,orient='index').head(2)

TypeError: object of type 'function' has no len()

In [16]:
cdr3_list=(df['aaSeqCDR3'])
type(cdr3_list)

pandas.core.series.Series

In [36]:
df_filtered=df[~df['aaSeqCDR3'].str.contains('_|\*', regex=True)]
df_filtered=df_filtered[['aaSeqCDR3', 'cloneCount']]

In [37]:
df_filtered.head()

,aaSeqCDR3,cloneCount
0,CASSRGTGSYEQYF,9936.0
1,CASSFDSGSYEQYF,6683.0
2,CASSPTGGGSGTEAFF,4793.0
3,CASSYLGRDEETQYF,3059.0
4,CASSLQGSQGNNQPQHF,3047.0


## Sample List

In [25]:
sample_list_batch1_batch2 = [
"CeD_KIR_NEG","CeD_KIR_POS","HD_KIR_NEG2","HD_KIR_NEG3",
"HD_KIR_NEG4","HD_KIR_NEG","HD_KIR_POS2","HD_KIR_POS3",
"HD_KIR_POS4","HD_KIR_POS","MS_KIR_NEG1","MS_KIR_NEG2",
"MS_KIR_NEG","MS_KIR_POS1","MS_KIR_POS2","MS_KIR_POS",
"SLE_KIR_NEG2","SLE_KIR_NEG3","SLE_KIR_NEG4","SLE_KIR_NEG",
"SLE_KIR_POS2","SLE_KIR_POS3","SLE_KIR_POS4","SLE_KIR_POS",
]

## Functions

In [18]:
def save_cdr3(input_sample):
    """Generate tables for cdr3 counts and save to sqlite db
    """
    df=pd.read_csv('/labs/mmdavis/chen/tcr/bulk_data_3/TRB/{}.TRB'.format(input_sample) ,sep='\t')
    df_filtered=df[~df['aaSeqCDR3'].str.contains('_|\*', regex=True)]
    df_filtered=df_filtered[['aaSeqCDR3', 'cloneCount']]
    conn=sqlite3.connect('/labs/mmdavis/chen/tcr/jing_TCR.db')
    df_filtered.to_sql("{}".format(input_sample), conn, if_exists="replace", index=False)
    conn.close()

#### Batch1 and Batch2 TCR

In [2]:
sample_list_batch1_batch2 = [
"CeD_KIR_NEG","CeD_KIR_POS","HD_KIR_NEG2","HD_KIR_NEG3",
"HD_KIR_NEG4","HD_KIR_NEG","HD_KIR_POS2","HD_KIR_POS3",
"HD_KIR_POS4","HD_KIR_POS","MS_KIR_NEG1","MS_KIR_NEG2",
"MS_KIR_NEG","MS_KIR_POS1","MS_KIR_POS2","MS_KIR_POS",
"SLE_KIR_NEG2","SLE_KIR_NEG3","SLE_KIR_NEG4","SLE_KIR_NEG",
"SLE_KIR_POS2","SLE_KIR_POS3","SLE_KIR_POS4","SLE_KIR_POS",
]

for i in sample_list_batch1_batch2:
    save_cdr3(i)

NameError: name 'sample_list' is not defined

#### Batch3 TCR

In [19]:
sample_list_batch3 = [
"SLE_KIR_POS7",
"SLE_KIR_NEG7",
"SLE_KIR_POS8",
"SLE_KIR_NEG8",
"HD_KIR_POS5",
"HD_KIR_NEG5",
"HD_KIR_POS6",
"HD_KIR_NEG6",
]

for i in sample_list_batch3:
    save_cdr3(i)

In [80]:
def count_v(input_sample):
    df=pd.read_csv('/labs/mmdavis/chen/tcr/TRB/{}.TRB'.format(input_sample) ,sep='\t')
    df_filtered=df[~df['aaSeqCDR3'].str.contains('_|\*', regex=True)]
    v_list=list(df_filtered['allVHitsWithScore'])
    
    count_v=dict()
    
    for i in v_list:
        a=i.split('*')[0]
        if a not in count_v:
            count_v[a]=1
        else:
            count_v[a]+=1
    df_V_count=pd.DataFrame.from_dict(count_v, orient='index',columns=[input_sample])
    return df_V_count

def count_v2(input_sample):
    df=pd.read_csv('/labs/mmdavis/chen/tcr/TRB/{}.TRB'.format(input_sample) ,sep='\t')
    df_filtered=df[~df['aaSeqCDR3'].str.contains('_|\*', regex=True)]
    v_list=list(df_filtered['allVHitsWithScore'])
    
    count_v=dict()
    
    for i in v_list:
        a=i.split('*')[0]
        if a not in count_v:
            count_v[a]=1
        else:
            count_v[a]+=1
    df_V_count=pd.DataFrame.from_dict(count_v, orient='index',columns=['count'])
    df_V_count.to_sql('{}_V_count'.format(input_sample), conn, if_exists = "replace", index = True)

def count_v_df_list():
    dfs=list()
    for i in sample_list:
        df=count_v(i)
        dfs.append(df)
    return dfs

for i in sample_list:
    count_v2(i)

In [31]:
#!jupyter nbconvert --to script JingLi_TCR.ipynb

In [55]:
dfs=count_v_df_list()

In [49]:
from functools import reduce
df_final = reduce(lambda left,right: pd.DataFrame.join(left,right), dfs)
df_final.head()

,CeD_KIR_NEG,CeD_KIR_POS,HD_KIR_NEG2,HD_KIR_NEG3,HD_KIR_NEG4,HD_KIR_NEG,HD_KIR_POS2,HD_KIR_POS3,HD_KIR_POS4,HD_KIR_POS,...,MS_KIR_POS2,MS_KIR_POS,SLE_KIR_NEG2,SLE_KIR_NEG3,SLE_KIR_NEG4,SLE_KIR_NEG,SLE_KIR_POS2,SLE_KIR_POS3,SLE_KIR_POS4,SLE_KIR_POS
TRBV5-4,420,219.0,420.0,563.0,370.0,663.0,228.0,204.0,151.0,203.0,...,39.0,47.0,472.0,602.0,324.0,650.0,178.0,283.0,159.0,94.0
TRBV7-6,716,447.0,458.0,1149.0,700.0,904.0,278.0,1035.0,207.0,411.0,...,118.0,73.0,662.0,674.0,509.0,1283.0,258.0,295.0,376.0,312.0
TRBV7-8,638,439.0,290.0,407.0,395.0,476.0,148.0,196.0,117.0,202.0,...,44.0,9.0,571.0,453.0,365.0,99.0,159.0,158.0,213.0,28.0
TRBV7-2,1380,985.0,585.0,746.0,667.0,1438.0,331.0,422.0,387.0,580.0,...,122.0,87.0,1035.0,738.0,553.0,422.0,405.0,251.0,286.0,292.0
TRBV5-6,801,338.0,329.0,908.0,670.0,724.0,269.0,408.0,107.0,297.0,...,122.0,59.0,845.0,582.0,509.0,380.0,425.0,270.0,238.0,36.0


In [67]:
df_final.to_sql('V_count', conn, if_exists = "replace", index = True)

## Analysis of V gene bias

In [68]:
import sqlite3
conn=sqlite3.connect('/labs/mmdavis/chen/tcr/jing_TCR.db')

In [69]:
V_count=pd.read_sql_query('select * from V_count', conn)

In [70]:
df=V_count.head()

In [73]:
df1=pd.melt(df, id_vars=['index'], value_vars=sample_list)

In [74]:
df1.head()

,index,variable,value
0,TRBV5-4,CeD_KIR_NEG,420.0
1,TRBV7-6,CeD_KIR_NEG,716.0
2,TRBV7-8,CeD_KIR_NEG,638.0
3,TRBV7-2,CeD_KIR_NEG,1380.0
4,TRBV5-6,CeD_KIR_NEG,801.0


In [ ]:
pd.melt(df, id_vars=['A'], value_vars=['B'])

In [63]:
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(2,3,1)
ax2 = fig.add_subplot(2,3,2)
ax3 = fig.add_subplot(2,3,3)
ax4 = fig.add_subplot(2,3,4)
ax5 = fig.add_subplot(2,3,5)
ax6 = fig.add_subplot(2,3,6)

# 10-01-19

In [ ]:
"5-2071unstim_S7_L001_R2_001.fastq.gz"
"5-2071unstim_S7_L001_R1_001.fastq.gz"
"3-1505KIR-CD8_S5_L001_R1_001.fastq.gz"
"3-1505KIR-CD8_S5_L001_R2_001.fastq.gz"
"7-2071KIR-CD8_S9_L001_R2_001.fastq.gz"
"7-2071KIR-CD8_S9_L001_R1_001.fastq.gz"
"2-1505noCD8_S4_L001_R2_001.fastq.gz"
"2-1505noCD8_S4_L001_R1_001.fastq.gz"
"6-2071noCD8_S8_L001_R1_001.fastq.gz"
"6-2071noCD8_S8_L001_R2_001.fastq.gz"
"1-1505unstim_S3_L001_R2_001.fastq.gz"
"1-1505unstim_S3_L001_R1_001.fastq.gz"
"8-2071KIRplusCD8_S10_L001_R2_001.fastq.gz"
"8-2071KIRplusCD8_S10_L001_R1_001.fastq.gz"
"4-1505KIRplusCD8_S6_L001_R2_001.fastq.gz"
"4-1505KIRplusCD8_S6_L001_R1_001.fastq.gz"

